# Word2Vec

Author : Nguyen Hoai Phuong

Date: June 13th 2021

## 1- Import necessary packages

In [56]:
import os
import pandas as pd
import string

import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

## 2- Load data

In [57]:
dataset = pd.read_csv('raw_data.csv')

In [58]:
dataset.head(10)

,content,category
0,Võ sĩ MMA bị knock-out lạ kỳ vì... 'đụng đầu' ...,Thể thao
1,"Thi đấu từ máy bay ở độ cao 6.096m, ngôi sao 6...",Thể thao
2,Ấn Độ yêu cầu mạng xã hội gỡ cụm từ 'biến thể ...,Công nghệ
3,Tay đua 19 tuổi Jason Dupasquier qua đời sau t...,Thể thao
4,Cảnh sát Ấn Độ 'cảnh báo' Công ty Twitter TTO ...,Công nghệ
5,Sủng Cỏ thiên đường hoang sơ nơi cửa vịnh Đà N...,Du lịch
6,"Giận liên đoàn, CĐV Indonesia không vào sân cổ...",Thể thao
7,Google bỏ cookie theo dõi người dùng là để 'là...,Công nghệ
8,"Võ sĩ hạng nặng đánh nhau trên sàn đấu, khán g...",Thể thao
9,Bánh trứng thời nghèo khó Dan Bing bỗng len lỏ...,Du lịch


In [59]:
dataset.describe()

,content,category
count,7172,7172
unique,7172,3
top,"Quýt hồng Lai Vung vào vụ chín, thu hút khách ...",Thể thao
freq,1,3058


## 3- Preprocessing data

### 3.1 - Cleanup text

In [60]:
document = dataset['content'].values
document

array(["Võ sĩ MMA bị knock-out lạ kỳ vì... 'đụng đầu' với đối thủ TTO - Trong lúc cố gắng tấn công, võ sĩ người Nga Kiril Fomenkov vô tình đụng đầu với đối thủ và bị knock-out một cách kỳ lạ ở trận so tài với Aleksandr Yakovenko ngày 30-5, tại sự kiện Eagle Fighting Championship của Nga.",
       'Thi đấu từ máy bay ở độ cao 6.096m, ngôi sao 6.000 lần nhảy dù tử nạn TTO - Ngôi sao nhảy dù 30 tuổi Dimitri Didenko đã tử nạn ngày 15-3, sau khi rơi từ độ cao 20.000 ft (6.096m) trong lúc đang thi đấu ở Giải vô địch nhảy dù Úc 2021. chiếc dù của anh gặp sự cố không thể mở ra được trong lúc thi đấu. Qua điều tra, cảnh sát Úc kết luận đây là một tai nạn và "gây ớn lạnh" cho tất cả những người đã chứng kiến. Cảnh sát Úc cho biết cụ thể: " Trong khoảng thời gian từ 13h chiều đến 14h30, người đàn ông đã nhảy từ một chiếc máy bay và người ta tin rằng chiếc dù của anh ấy đã không được mở đúng cách khiến anh ấy rơi xuống đất. Anh ấy bị thương rất nặng và chết tại hiện trường, dù đội cứu thương đã cố

In [111]:
raw_data = []
for line in document:
    s = line.lower().replace('\n', ' ')
    s = s.replace('"', ' ')
    s = s.replace("'", ' ')
    raw_data.append(s)

### 3.2 - Tokenize

In this step, we will tokenize the vietnamese text. In order to do that, please following this [link](https://github.com/vncorenlp/VnCoreNLP#python) to install vncorenlp and running the server.

After running the vncorenlp server on localhost:8000, we can use it to tokenize our text

In [105]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP(address="http://127.0.0.1", port=8000)

** Example of how to use vncorenlp to tokenize text **

In [106]:
# Input 
text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."

# To perform word segmentation, POS tagging, NER and then dependency parsing
annotated_text = annotator.annotate(text)
print(annotated_text)

print('------------------------------------------------------------------')

# To perform word segmentation only
word_segmented_text = annotator.tokenize(text)
print(word_segmented_text)

{'sentences': [[{'index': 1, 'form': 'Ông', 'posTag': 'Nc', 'nerLabel': 'O', 'head': 4, 'depLabel': 'sub'}, {'index': 2, 'form': 'Nguyễn_Khắc_Chúc', 'posTag': 'Np', 'nerLabel': 'B-PER', 'head': 1, 'depLabel': 'nmod'}, {'index': 3, 'form': 'đang', 'posTag': 'R', 'nerLabel': 'O', 'head': 4, 'depLabel': 'adv'}, {'index': 4, 'form': 'làm_việc', 'posTag': 'V', 'nerLabel': 'O', 'head': 0, 'depLabel': 'root'}, {'index': 5, 'form': 'tại', 'posTag': 'E', 'nerLabel': 'O', 'head': 4, 'depLabel': 'loc'}, {'index': 6, 'form': 'Đại_học', 'posTag': 'N', 'nerLabel': 'B-ORG', 'head': 5, 'depLabel': 'pob'}, {'index': 7, 'form': 'Quốc_gia', 'posTag': 'N', 'nerLabel': 'I-ORG', 'head': 6, 'depLabel': 'nmod'}, {'index': 8, 'form': 'Hà_Nội', 'posTag': 'Np', 'nerLabel': 'I-ORG', 'head': 6, 'depLabel': 'nmod'}, {'index': 9, 'form': '.', 'posTag': 'CH', 'nerLabel': 'O', 'head': 4, 'depLabel': 'punct'}], [{'index': 1, 'form': 'Bà', 'posTag': 'Nc', 'nerLabel': 'O', 'head': 9, 'depLabel': 'sub'}, {'index': 2, 'for

** Tokenized the document **

In [114]:
train_data = []
for line in raw_data:
    word_segmented_text = annotator.tokenize(line)
    train_data.extend(word_segmented_text)

## Train model

In [115]:
# Skipgram model
model = Word2Vec(train_data, min_count = 1, window = 5, sg = 1)
model.wv.save('word2vec_skipgram.model')

# CBOW model
model = Word2Vec(train_data, min_count = 1, window = 5, sg = 0)
model.wv.save('word2vec_CBOW.model')

## Test skipgram result

In [120]:
model = gensim.models.KeyedVectors.load('word2vec_skipgram.model')

In [121]:
model.most_similar(positive=['du_lịch'],topn=10)

[('kích_cầu', 0.7140858173370361),
 ('nội_địa', 0.7095187306404114),
 ('lữ_hành', 0.6995542645454407),
 ('sông_cửu_long', 0.6848586201667786),
 ('mở_cửa', 0.6757857799530029),
 ('tour', 0.673807680606842),
 ('tham_quan', 0.6729914546012878),
 ('hội_chợ', 0.6717577576637268),
 ('diều', 0.6709558367729187),
 ('định_hướng', 0.6647369265556335)]

In [122]:
model.most_similar(positive=['nishino'],topn=10)

[('akira', 0.9678785800933838),
 ('riedl', 0.9642030596733093),
 ('yong', 0.9608131051063538),
 ('jin', 0.9606932997703552),
 ('mcmenemy', 0.960474967956543),
 ('shin', 0.9597348570823669),
 ('joachim', 0.9561440348625183),
 ('tae', 0.9541601538658142),
 ('loew', 0.9528109431266785),
 ('toshiya', 0.9526103734970093)]

## Test cbow result

In [123]:
model1 = gensim.models.KeyedVectors.load('word2vec_CBOW.model')

In [124]:
model1.most_similar(positive=['du_lịch'],topn=10)

[('quốc_tế', 0.8715706467628479),
 ('lữ_hành', 0.8506467342376709),
 ('hội_nhập', 0.8504394292831421),
 ('phát_triển', 0.8407766222953796),
 ('ngành', 0.8349848389625549),
 ('văn_hoá', 0.8338674902915955),
 ('tham_quan', 0.8241179585456848),
 ('chương_trình', 0.8218135237693787),
 ('ẩm_thực', 0.8210338950157166),
 ('tour', 0.8206571340560913)]

In [125]:
model1.most_similar(positive=['nishino'],topn=10)

[('klopp', 0.9610585570335388),
 ('guillaume', 0.9599000215530396),
 ('roy', 0.9558120369911194),
 ('capello', 0.9519848823547363),
 ('akira', 0.9502278566360474),
 ('mourinho', 0.9492140412330627),
 ('hodgson', 0.9474775791168213),
 ('jurgen', 0.9461413621902466),
 ('toshiya', 0.945853054523468),
 ('miura', 0.9445356130599976)]